# Implementation of Dueling Network Architecture For RL
# https://arxiv.org/pdf/1511.06581.pdf

In [33]:
import numpy as np
import random
import gym

In [34]:
from numpy import tanh
from scipy.special import softmax
import pdb
from collections import deque

In [35]:
class DQNNModel(object):
    '''
    Two layer neural net
    input X
    Z1 = dot(X, W1) + B1
    Z2 = tanh(Z1)
    
    #Advantage
    Z3_A = dot(Z2, W2_A) + B2_A
    y_hat_a = Z3_A
    
    #Value
    Z3_V = dot(Z2, W2_V) + B2_V
    y_hat_v = Z3_V
    
    Loss(L) = 0.5 * ( (y_hat_a+y_hat_v) - (y_a+y_v) )**2
    
    dL/dy_hat_a = dlogp_a = y_hat_a + y_hat_v - y_a - y_v
    dL/dy_hat_v = dlogp_v = y_hat_a + y_hat_v - y_a - y_v
    
    dy_hat_a/dZ3_A = 1
    dZ3_A/dW2_A = Z2
    dZ3_A/dB2_A = 1
    
    dZ3_A/dZ2 = W2_A
    dZ2/dZ1 = 1 - Z2**2
    dZ1/dW1 = X
    dZ1/dB1 = 1
    
    dy_hat_v/dZ3_V = 1
    dZ3_V/dW2_V = Z2
    dZ3_V/dB2_V = 1
    
    dZ3_V/dZ2 = W2_V
    dZ2/dZ1 = 1 - Z2**2
    dZ1/dW1 = X
    dZ1/dB1 = 1
    
    
    
    Using chain rule
    
    dL/dW2_A = (dlogp_a)*Z2
    dL/dB2_A = dlogp_a
    
    dL/dW1 = (dlogp_a)*W2_A*(1-Z2**2)*X
    (input,hidden) = (1,output)*(hidden,output)*(1,hidden)*(1,input)
    dL/dB1 = (dlogp_a)*W2_A*(1-Z2**2)
    
    dL/dW2_V = (dlogp_v)*Z2
    dL/dB2_V = dlogp_v
    
    dL/dW1 = (dlogp_v)*W2_V*(1-Z2**2)*X
    dL/dB1 = (dlogp_v)*W2_V*(1-Z2**2)
    
    
    '''
    def __init__(self, ninput, nhidden, noutput):
        self.ninput_ = ninput
        self.nhidden_ = nhidden
        self.noutput_ = noutput
        #initialize model weights
        #X=(1,input)
        self.W1_ = np.random.randn(ninput, nhidden) / np.sqrt(nhidden)
        self.B1_ = np.random.randn(1, nhidden) / np.sqrt(nhidden)
        
        self.W2_A_ = np.random.randn(nhidden, noutput) / np.sqrt(noutput)
        self.B2_A_ = np.random.randn(1, noutput) / np.sqrt(noutput)
        
        self.W2_V_ = np.random.randn(nhidden, 1) / np.sqrt(nhidden)
        self.B2_V_ = np.random.randn(1, 1)
        
        
        self.dW1_cache_ = 0.
        self.dB1_cache_ = 0.
        self.dW2_A_cache_ = 0.
        self.dB2_A_cache_ = 0.
        self.dW2_V_cache_ = 0.
        self.dB2_V_cache_ = 0.

        
        
    def __str__(self):
        return_str = ""
        for i,j in self.__dict__.items():
            return_str += str(i) + " " + str(j) + "\n"  
        return return_str
    
    def forward_adv(self, x):
        #x=(1,input)
        #W1=(input,hidden)
        z1 = np.dot(x, self.W1_) + self.B1_ #(1,hidden)
        z2 = tanh(z1) #(1,hidden)
        
        z3_a = np.dot(z2, self.W2_A_) + self.B2_A_ #(1,output)
        #W2=(hidden, output)
        y_hat_adv = z3_a #(1,output)
        return y_hat_adv, z1, z2, z3_a, self.W1_, self.W2_A_

    def forward_val(self, x):
        #x=(1,input)
        #W1=(input,hidden)
        z1 = np.dot(x, self.W1_) + self.B1_ #(1,hidden)
        z2 = tanh(z1) #(1,hidden)
        
        z3_v = np.dot(z2, self.W2_V_) + self.B2_V_ #(1,output)
        #W2=(hidden, output)
        y_hat_val = z3_v #(1,1)
        return y_hat_val, z1, z2, z3_v, self.W1_, self.W2_V_
    
    
    def loss(self, y, y_hat):
        loss = 0.5 * (y_hat - y)**2
        return loss
    
    def backward(self, x, y_a, y_v, y_hat_a, y_hat_v, z1, z2, z3_v, z3_a, w1, w2_a, w2_v, clip=True):
        #pdb.set_trace()
        #1.5 multiplier comes from the fact that we subtract mean from y_hat_v and we have 2 actions
        dlogp_a = 1.5 * ( (y_hat_a+y_hat_v)-(y_a+y_v) )
        
        dlogp_v = 1.5 * ( (y_hat_v+y_hat_a).mean(axis=1)-(y_v+y_a).mean(axis=1) )
        
        dlogp_v = np.reshape(dlogp_v, y_v.shape)
        
        if clip:
            np.clip(dlogp_a, -1., 1., out=dlogp_a)
            np.clip(dlogp_v, -1., 1., out=dlogp_v)
            #dlogp_v = max(-1., min(1.,dlogp_v))
        
        dB2_A = dlogp_a # (1, output)
        dW2_A = np.dot(z2.T, dlogp_a) #(hidden, output)
        
        dB2_V = dlogp_v
        dW2_V = np.dot(z2.T, dlogp_v)
        
        dB1_a = np.dot(dlogp_a, w2_a.T)*(1-(z2*z2)) #(1,hidden)
        dW1_a = np.dot(x.T, dB1_a) #(input,hidden)

        dB1_v = np.dot(dlogp_v, w2_v.T)*(1-(z2*z2)) #(1,hidden)
        dW1_v = np.dot(x.T, dB1_v) #(input,hidden)
        
        dB1 = dB1_a + dB1_v
        dW1 = dW1_a + dW1_v
        
        return dW1, dB1, dW2_A, dB2_A, dW2_V, dB2_V
            
    
    def update_SGD(self, dW1, dB1, dW2_A, dB2_A, dW2_V, dB2_V, lr=1e-4):
        self.W1_ -= lr*dW1
        self.B1_ -= lr*dB1
        
        self.W2_A_ -= lr*dW2_A
        self.B2_A_ -= lr*dB2_A

        self.W2_V_ -= lr*dW2_V
        self.B2_V_ -= lr*dB2_V
        
    
    def put_from_model(self, model):
        self.W1_ = model.W1_
        self.B1_ = model.B1_
        self.W2_A_ = model.W2_A_
        self.B2_A_ = model.B2_A_
        self.W2_V_ = model.W2_V_
        self.B2_V_ = model.B2_V_

       
    def update_RMSProp(self, dW1, dB1, dW2_A, dB2_A, dW2_V, dB2_V, lr=1e-4):
        decay_rate = 0.9
        eps = 1e-4
        
        self.dW1_cache_ = decay_rate * self.dW1_cache_ + (1 - decay_rate) * (dW1)**2
        self.W1_ += - lr * dW1 / (np.sqrt(self.dW1_cache_) + eps)

        self.dB1_cache_ = decay_rate * self.dB1_cache_ + (1 - decay_rate) * (dB1)**2
        self.B1_ += - lr * dB1 / (np.sqrt(self.dB1_cache_) + eps)

        self.dW2_A_cache_ = decay_rate * self.dW2_A_cache_ + (1 - decay_rate) * (dW2_A)**2
        self.W2_A_ += - lr * dW2_A / (np.sqrt(self.dW2_A_cache_) + eps)

        self.dB2_A_cache_ = decay_rate * self.dB2_A_cache_ + (1 - decay_rate) * (dB2_A)**2
        self.B2_A_ += - lr * dB2_A / (np.sqrt(self.dB2_A_cache_) + eps)        

        self.dW2_V_cache_ = decay_rate * self.dW2_V_cache_ + (1 - decay_rate) * (dW2_V)**2
        self.W2_V_ += - lr * dW2_V / (np.sqrt(self.dW2_V_cache_) + eps)

        self.dB2_V_cache_ = decay_rate * self.dB2_V_cache_ + (1 - decay_rate) * (dB2_V)**2
        self.B2_V_ += - lr * dB2_V / (np.sqrt(self.dB2_V_cache_) + eps)        
        

In [36]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

In [37]:
state = env.reset()
INPUT_UNITS = state.shape[0]
HIDDEN_UNITS = 100
OUTPUT_UNITS = env.action_space.n
model = DQNNModel(INPUT_UNITS, HIDDEN_UNITS, OUTPUT_UNITS)
target_model = DQNNModel(INPUT_UNITS, HIDDEN_UNITS, OUTPUT_UNITS)


In [ ]:
num_episodes = 12000

experience_replay_size = 250
experience_replay_queue_max = 10000

copy_model_after = 5
copy_model_counter = 0

display_after = 20
display_after_counter = 0

epsilon = 1
epsilon_min = 0.01
epsilon_decay = 0.995

total_steps = 0
total_episode_steps_rs = []
episode_steps_rs = []
batch_loss_rs = []

replay_queue = deque(maxlen=experience_replay_queue_max)

for episode in range(num_episodes):
    episode_steps = 0
    while True:
        state = state.reshape(1,-1)
        y_hat_v, _, _, _, _, _ = model.forward_val(state)
        y_hat_a, _, _, _, _, _ = model.forward_adv(state)
        
        y_hat = y_hat_v + (y_hat_a - y_hat_a.mean(axis=1))
        action_values = y_hat
        actions_sm = softmax(action_values)
        
        #exploration vs exploitation
        if epsilon > random.random():
            action = 0 if 0.5>random.random() else 1
        else:
            action = 0 if actions_sm[0][0]>random.random() else 1        
        
        new_state, reward, done, info = env.step(action)
        episode_steps += 1
        
        replay_queue.append([state,action,reward-done,new_state,done])

        state = new_state
        total_steps += 1

        if (total_steps%experience_replay_size == 0):
            batch_loss = 0.
            batch = random.sample(list(replay_queue), experience_replay_size)
            
            dW1, dB1, dW2, dB2 = 0.,0.,0.,0.
            for i,obs in enumerate(batch):
                s, a, r, n_s, d = obs
                
                n_s = n_s.reshape(1,-1)
                
                y_a = np.zeros((1,env.action_space.n))
                
                if d:
                    y_a[0][a] = r
                    y_v = np.array([[0.]])
                    
                else:
                    y_a[0][a] = r

                    t_val_v,_,_,_,_,_ = target_model.forward_val(n_s)
                    t_val_a,_,_,_,_,_ = target_model.forward_adv(n_s)
                    
                    t_val_a -= t_val_a.mean(axis=1)
                    
                    y_a += 0.99 * (t_val_a)
                    y_v = 0.99*t_val_v
                    

                y_hat_v, z1, z2, z3_v, w1, w2_v = model.forward_val(s)
                                
                y_hat_a, z1, z2, z3_a, w1, w2_a = model.forward_adv(s)
                y_hat_a -= y_hat_a.mean(axis=1)
                
                #pdb.set_trace()
                dw1, db1, dw2_a, db2_a, dw2_v, db2_v = model.backward(s, y_a,y_v, y_hat_a, y_hat_v, z1, z2, z3_v, z3_a, w1, w2_a, w2_v)
                                
                #dW1 += dw1
                #dB1 += db1
                #dW2 += dw2
                #dB2 += db2
                model.update_SGD(dw1, db1, dw2_a, db2_a, dw2_v, db2_v)
                #model.update_RMSProp(dw1, db1, dw2_a, db2_a, dw2_v, db2_v)
                batch_loss += model.loss(y_a+y_v, y_hat_a+y_hat_v).mean()
            
            
            #model.update_SGD(dW1, dB1, dW2, dB2)
            #update target model with model
            copy_model_counter += 1
            if(copy_model_counter%copy_model_after == 0):
                target_model.put_from_model(model)
            
            average_loss = batch_loss / len(batch)                         
            batch_loss_rs.append(average_loss)
            display_after_counter += 1

            if epsilon > epsilon_min:
                epsilon *= epsilon_decay
            
            if (display_after_counter%display_after == 0):
                episode_steps_buffer = episode_steps_rs
                episode_loss_buffer = batch_loss_rs[-display_after:]
                #print(episode_loss_buffer)
                min_steps = np.min(episode_steps_buffer)
                max_steps = np.max(episode_steps_buffer)
                avg_steps = np.mean(episode_steps_buffer)
                print("Episode %d, batch episodes %d, max %d, min %d, avg %d steps, avg loss %f"%(episode+1, len(episode_steps_rs),max_steps, 
                                                                                                  min_steps, avg_steps, np.mean(episode_loss_buffer)))
                episode_steps_rs = []
                                
        if done:
            state = env.reset()
            episode_steps_rs.append(episode_steps)
            total_episode_steps_rs.append(episode_steps)
            break
            

Episode 223, batch episodes 222, max 71, min 9, avg 22 steps, avg loss 0.344705
Episode 437, batch episodes 214, max 70, min 8, avg 23 steps, avg loss 0.741256
Episode 695, batch episodes 258, max 99, min 9, avg 19 steps, avg loss 3.254281
Episode 995, batch episodes 300, max 52, min 8, avg 16 steps, avg loss 9.829246
Episode 1296, batch episodes 301, max 47, min 8, avg 16 steps, avg loss 21.273506
Episode 1648, batch episodes 352, max 37, min 8, avg 14 steps, avg loss 37.100487
Episode 2004, batch episodes 356, max 44, min 8, avg 14 steps, avg loss 60.907460
Episode 2385, batch episodes 381, max 44, min 8, avg 13 steps, avg loss 69.874365
Episode 2774, batch episodes 389, max 32, min 8, avg 12 steps, avg loss 25.581501
Episode 3136, batch episodes 362, max 36, min 8, avg 13 steps, avg loss 20.004325
Episode 3323, batch episodes 187, max 82, min 12, avg 26 steps, avg loss 27.993838
Episode 3424, batch episodes 101, max 153, min 16, avg 49 steps, avg loss 19.351142
Episode 3516, batch e

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.scatter(np.array(range(len(total_episode_steps_rs))), total_episode_steps_rs)

In [ ]:
import pandas as pd
s1 = pd.Series(total_episode_steps_rs).rolling(20).mean()
s1.dropna(inplace=True)
plt.plot(s1)

In [ ]:
import pandas as pd
s2 = pd.Series(total_episode_steps_rs).rolling(50).mean()
s2.dropna(inplace=True)
plt.plot(s2)